In [1]:
# Import necessary libraries
import pandas as pd
import re
import unicodedata
from google.colab import files

# Function to clean text by removing punctuation, converting to lowercase, and removing diacritics
def clean_text_for_matching(text):
    """
    Cleans the input text for matching by removing punctuation, converting to lowercase,
    and removing diacritics (accents).
    """
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation using regex
    text = text.lower()  # Convert to lowercase
    # Remove diacritics
    text = ''.join(
        char for char in unicodedata.normalize('NFD', text)
        if unicodedata.category(char) != 'Mn'
    )
    return text

# Upload the lexicon and test corpus files
print("Please upload your lexicon Excel file (e.g., 'corrected_lexicon.xlsx'):")
uploaded_lexicon = files.upload()

print("Please upload your testing corpus Excel file (e.g., 'test_corpus.xlsx'):")
uploaded_test_corpus = files.upload()

# Load the uploaded lexicon file
lexicon_file = list(uploaded_lexicon.keys())[0]
df = pd.read_excel(lexicon_file)

# Load the uploaded test corpus file
test_corpus_file = list(uploaded_test_corpus.keys())[0]
test_corpus_df = pd.read_excel(test_corpus_file)

# Adjust column names for consistency and include 'sepedi'
df.columns = ['ciluba', 'french', 'english', 'afrikaans', 'zulu', 'sepedi', 'score', 'sentiment', 'nature']

# Define language mapping for consistency, adding 'sepedi'
language_mapping = {
    'francais': 'french',
    'afrikaans': 'afrikaans',
    'zulu': 'zulu',
    'ciluba': 'ciluba',
    'english': 'english',
    'sepedi': 'sepedi'
}

# Function to preprocess the lexicon and calculate average sentiment scores for each word/phrase
def create_multi_word_translation_dict_with_sentiment(df):
    """
    Creates a translation dictionary that includes both single words and multi-word entries
    along with their average sentiment scores.
    """
    # Group by each language and English to calculate the average sentiment score
    sentiment_averages = {}
    all_sentiments = {}
    for lang in ['french', 'afrikaans', 'zulu', 'ciluba', 'sepedi', 'english']:
        sentiment_averages[lang] = df.groupby(lang)['score'].mean().to_dict()
        all_sentiments[lang] = df.groupby(lang)['score'].apply(list).to_dict()

    translation_dicts = {
        'french': {'english': dict(zip(df['french'].str.lower(), df['english'].str.lower()))},
        'afrikaans': {'english': dict(zip(df['afrikaans'].str.lower(), df['english'].str.lower()))},
        'zulu': {'english': dict(zip(df['zulu'].str.lower(), df['english'].str.lower()))},
        'ciluba': {'english': dict(zip(df['ciluba'].str.lower(), df['english'].str.lower()))},
        'sepedi': {'english': dict(zip(df['sepedi'].str.lower(), df['english'].str.lower()))},
        'english': {
            'french': dict(zip(df['english'].str.lower(), df['french'].str.lower())),
            'afrikaans': dict(zip(df['english'].str.lower(), df['afrikaans'].str.lower())),
            'zulu': dict(zip(df['english'].str.lower(), df['zulu'].str.lower())),
            'ciluba': dict(zip(df['english'].str.lower(), df['ciluba'].str.lower())),
            'sepedi': dict(zip(df['english'].str.lower(), df['sepedi'].str.lower()))
        }
    }

    # Add multi-word entries with their sentiment averages
    for lang in ['french', 'afrikaans', 'zulu', 'ciluba', 'sepedi']:
        multi_word_entries = df[df[lang].str.contains(' ') & df['english'].str.contains(' ', na=False)]
        for _, row in multi_word_entries.iterrows():
            source_phrase = row[lang].lower()
            english_phrase = row['english'].lower()
            translation_dicts[lang]['english'][source_phrase] = english_phrase
            translation_dicts['english'][lang][english_phrase] = source_phrase

    return translation_dicts, sentiment_averages, all_sentiments

# Create the translation dictionaries with sentiment support
translation_dicts, sentiment_averages, all_sentiments = create_multi_word_translation_dict_with_sentiment(df)

# Function for the new sentiment analysis method based on the given logic
def compute_sentiment_v2(scores):
    """
    Computes sentiment based on the rules:
    - If one score, return it.
    - If two scores, return the extreme value.
    - If three or more scores, return the average of the majority group (positive or negative).
    """
    if len(scores) == 1:
        return scores[0]
    elif len(scores) == 2:
        return max(scores, key=abs)
    else:
        positive_scores = [s for s in scores if s > 0]
        negative_scores = [s for s in scores if s < 0]
        if len(positive_scores) >= len(negative_scores):
            return sum(positive_scores) / len(positive_scores) if positive_scores else 0
        else:
            return sum(negative_scores) / len(negative_scores) if negative_scores else 0

# Function to translate and compute sentiment based on two methods, including word scores
def translate_and_analyze_sentiments(text, source_lang, target_lang, translation_dicts, sentiment_averages, all_sentiments):
    source_lang = language_mapping.get(source_lang.lower(), source_lang.lower())
    target_lang = language_mapping.get(target_lang.lower(), target_lang.lower())

    cleaned_text = clean_text_for_matching(text)
    source_to_english = translation_dicts.get(source_lang, {}).get('english', {})
    english_to_target = translation_dicts.get('english', {}).get(target_lang, {})
    source_sentiments_avg = sentiment_averages.get(source_lang, {})
    source_sentiments_all = all_sentiments.get(source_lang, {})

    words = cleaned_text.split()
    translated_sentence = []
    total_score_avg = 0
    total_score_v2 = 0
    word_scores_avg = []  # List to store "word:score" strings
    word_scores_v2 = []   # List to store "word:score" strings
    i = 0

    while i < len(words):
        max_match_length = 0
        matched_phrase = None
        translated_phrase = None
        phrase_scores = []

        # Attempt to match the longest possible phrase
        for j in range(i + 1, len(words) + 1):
            phrase = ' '.join(words[i:j])
            if phrase in source_to_english:
                max_match_length = j - i
                matched_phrase = phrase
                translated_phrase = english_to_target.get(source_to_english[phrase], source_to_english[phrase])
                phrase_scores = source_sentiments_all.get(matched_phrase, [])

        if matched_phrase:
            translated_sentence.append(str(translated_phrase) if translated_phrase else "")
            avg_score = source_sentiments_avg.get(matched_phrase, 0)
            v2_score = compute_sentiment_v2(phrase_scores)
            total_score_avg += avg_score
            total_score_v2 += v2_score
            word_scores_avg.append(f"{matched_phrase}:{avg_score}")
            word_scores_v2.append(f"{matched_phrase}:{v2_score}")
            i += max_match_length
        else:
            word = words[i]
            translated_word = english_to_target.get(source_to_english.get(word, word), source_to_english.get(word, word))
            word_scores = source_sentiments_all.get(word, [])
            avg_score = source_sentiments_avg.get(word, 0)
            v2_score = compute_sentiment_v2(word_scores)
            translated_sentence.append(str(translated_word) if translated_word else "")
            total_score_avg += avg_score
            total_score_v2 += v2_score
            word_scores_avg.append(f"{word}:{avg_score}")
            word_scores_v2.append(f"{word}:{v2_score}")
            i += 1

    translated_text = ' '.join(translated_sentence)
    sentiment_avg = "neutral"
    sentiment_v2 = "neutral"

    if total_score_avg > 0.05:
        sentiment_avg = "positive"
    elif total_score_avg < -0.05:
        sentiment_avg = "negative"

    if total_score_v2 > 0.05:
        sentiment_v2 = "positive"
    elif total_score_v2 < -0.05:
        sentiment_v2 = "negative"

    # Join word scores into a single string separated by semicolons
    word_scores_avg_str = '; '.join(word_scores_avg)
    word_scores_v2_str = '; '.join(word_scores_v2)

    # Return all required fields
    return translated_text, total_score_avg, word_scores_avg_str, sentiment_avg, total_score_v2, word_scores_v2_str, sentiment_v2

# Function to safely translate and analyze sentiment for both methods
def safe_translate_and_analyze_sentiments(row, translation_dicts, sentiment_averages, all_sentiments):
    text = row['sentence']
    source_lang = row['source_language']
    target_lang = row['target_language']
    if isinstance(text, str) and isinstance(source_lang, str) and isinstance(target_lang, str):
        return translate_and_analyze_sentiments(text, source_lang, target_lang, translation_dicts, sentiment_averages, all_sentiments)
    else:
        # Return default values if inputs are not valid
        return "", 0, "", "neutral", 0, "", "neutral"

# Apply the sentiment analysis to the test corpus DataFrame
test_corpus_df['translated_text'], \
test_corpus_df['total_score_avg'], \
test_corpus_df['word_scores_avg'], \
test_corpus_df['sentiment_avg'], \
test_corpus_df['total_score_v2'], \
test_corpus_df['word_scores_v2'], \
test_corpus_df['sentiment_v2'] = zip(*test_corpus_df.apply(
    lambda row: safe_translate_and_analyze_sentiments(
        row,
        translation_dicts,
        sentiment_averages,
        all_sentiments
    ), axis=1
))

# Define the desired column order
desired_order = [
    'sentence',
    'source_language',
    'target_language',
    'total_score_avg',
    'word_scores_avg',
    'sentiment_avg',
    'total_score_v2',
    'word_scores_v2',
    'sentiment_v2',
    'translated_text'
]

# Reorder the DataFrame columns accordingly
test_corpus_df = test_corpus_df[desired_order + [col for col in test_corpus_df.columns if col not in desired_order]]

# Display the updated DataFrame with the translations and sentiment analysis
test_corpus_df.head(20)



Please upload your lexicon Excel file (e.g., 'corrected_lexicon.xlsx'):


Saving corrected_lexicon.xlsx to corrected_lexicon.xlsx
Please upload your testing corpus Excel file (e.g., 'test_corpus.xlsx'):


Saving test_corpus.xlsx to test_corpus.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Optionally, export the DataFrame to a CSV file
# Uncomment the following lines if you wish to download the results

test_corpus_df.to_csv('sentiment_analysis_results.csv', index=False)
files.download('sentiment_analysis_results.csv')


In [3]:
!pip install sweetviz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 57.7 MB/s eta 0:00:00


In [4]:
import pandas as pd
import sweetviz as sv

# Load your corrected lexicon dataset
lexicon_df = pd.read_excel('/content/corrected_lexicon.xlsx')

# Generate the Sweetviz report
report = sv.analyze(lexicon_df)

# Output the report as an HTML file
report.show_html('report.html')


                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
